In [43]:
# Build image
!docker build . --quiet --tag code-scanner:latest

sha256:f6fbd116e047632f75e2cc0e74b93548bd3941f178eca5af6381defbdbde83be


In [40]:
# Run container
!docker run -d -p 8000:8000 --name test-code-scanner -e AWS_REGION=us-west-2 -e AWS_DEFAULT_REGION=us-west-2 code-scanner:latest --host 0.0.0.0 --port 8000

d2e20c99da7e80f5b8ae0592dc7dc7311589440e9dc63b7153a9b1d4d1b6244f


In [24]:
# check container logs
!docker logs test-code-scanner

In [38]:
# kill and remove container
!docker kill test-code-scanner && docker rm test-code-scanner

test-code-scanner
test-code-scanner


In [13]:
# list contexts to get docker host DNS
!docker context ls

NAME                             DESCRIPTION                               DOCKER ENDPOINT                                       ERROR
default                          Current DOCKER_HOST based configuration   unix:///var/run/docker.sock                           
m5.large_i-00afbfba37597d9f2 *                                             tcp://ip-10-3-1-105.us-west-2.compute.internal:1111   


In [14]:
# IP from docker context when using Studio Docker CLI extension
host_ip = "ip-10-3-1-105.us-west-2.compute.internal"

In [15]:
%%bash -s $host_ip
curl ${1}:8000/ping

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    22  100    22    0     0   3070      0 --:--:-- --:--:-- --:--:--  3142


{"Health_Check":"200"}

In [16]:
%%bash -s $host_ip
curl -X POST ${1}:8000/scan -d '{"script": "\nimport boto3\n\nsm_client = boto3.client(\"sagemaker\")\nresponse = sm_client.list_models()\n\nprint(response[\"Models\"])\n", "language": "Python1"}'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   538  100   375  100   163  99522  43259 --:--:-- --:--:-- --:--:--  175k


[{"error":{},"stacktrace":"Traceback (most recent call last):\n  File \"/code/app/scanners/base.py\", line 26, in scan\n    self.validate(language)\n  File \"/code/app/scanners/base.py\", line 22, in validate\n    raise f\"UnSupported programming language {language}, supported languages are {self.file_ext.keys()}\"\nTypeError: exceptions must derive from BaseException\n"}]

In [41]:
%%bash -s $host_ip
curl -X POST ${1}:8000/scan -d '{"script": "import os\nuser_id = input(\"enter some text\")\nos.system(f\"echo {user_id}\")", "language": "Python", "scanner": "codeguru"}'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1469  100  1331  100   138    119     12  0:00:11  0:00:11 --:--:--   341


[{"Start line":3,"End line":3,"Recommendations":"Passing user-provided input to Python subprocess, OS, and command functions without validation or sanitization makes your code vulnerable to running arbitrary OS commands. To prevent this, implement input validation and use secure functions. For more information, see [CWE-77](https://cwe.mitre.org/data/definitions/77.html), [CWE-78](https://cwe.mitre.org/data/definitions/78.html) and [CWE-88](https://cwe.mitre.org/data/definitions/88.html)\n\nThis line of code makes an outdated API call to start and communicate with processes. We recommend that you use the `subprocess` module to start new processes, connect with their pipes, and get their return codes.\n\n[Learn more](https://docs.python.org/3/library/subprocess.html)\n\nStarting a process with a shell, possible injection detected, security issue. https://bandit.readthedocs.io/en/latest/plugins/b605_start_process_with_a_shell.html\n\n"},{"Start line":2,"End line":2,"Recommendations":"The

In [42]:
%%bash -s $host_ip
curl -X POST ${1}:8000/scan -d '{"script": "import os\nuser_id = input(\"enter some text\")\nos.system(f\"echo {user_id}\")", "language": "Python", "scanner": "semgrep"}'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   139    0     2  100   137      0     18  0:00:07  0:00:07 --:--:--     0


[]

In [133]:
import os
user_id = input("enter some text")
os.system(f"echo {user_id}")

enter some text Hello


Hello


0

In [10]:
!aws ecr get-login-password --region us-west-2 | docker login --username AWS --password-stdin 871623330196.dkr.ecr.us-west-2.amazonaws.com

WARNING! Your password will be stored unencrypted in /root/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [11]:
!docker tag code-scanner:latest 871623330196.dkr.ecr.us-west-2.amazonaws.com/code-scanner:latest

In [12]:
!docker push 871623330196.dkr.ecr.us-west-2.amazonaws.com/code-scanner:latest

The push refers to repository [871623330196.dkr.ecr.us-west-2.amazonaws.com/code-scanner]

f989d2d8: Preparing 
0a992372: Preparing 
1d97a959: Preparing 
9e73a2bd: Preparing 
b33ac6be: Preparing 
6509a367: Preparing 
ddf37fbf: Preparing 
73b3aeff: Preparing 
1311ca21: Preparing 
5d79bb3c: Preparing 
00162992: Preparing 
5dd59276: Preparing 
ddf37fbf: Waiting g 
f2d28c7e: Preparing 
e8af23e5: Preparing 
07b77614: Preparing 
a1c19ab6: Preparing 
0e682b9d: Preparing 
5304b3c1: Preparing 
906b5625: Preparing 
0c84bfbe: Preparing 
dd59276: Pushed   152.5MB/152.5MBlatest: digest: sha256:45b8bf362924d729111bcf801499a23701d326ed3c5c647e5a5a84677a6d968b size: 4912


In [2]:
!docker pull returntocorp/semgrep

Using default tag: latest
latest: Pulling from returntocorp/semgrep

a8db6415: Pulling fs layer 
d5e70e42: Pulling fs layer 
a579cd50: Pulling fs layer 
8c10ee34: Pulling fs layer 
b3fa61f8: Pulling fs layer 
3b615cde: Pulling fs layer 
59e5b97d: Pulling fs layer 
583c3bde: Pulling fs layer 
bdb8814b: Pulling fs layer 
596a008a: Pulling fs layer 
f0b32f57: Pulling fs layer 
0a81459b: Pulling fs layer 
27999d47: Pulling fs layer 
b8ca7cf4: Pulling fs layer 
46bf2140: Pulling fs layer 
40b4f99a: Pull complete  200B/200B7kBBDigest: sha256:b4637a27abf1e49aeab753fdfd41d917493d8fc7f6da9de2daf109b4d7369fe8
Status: Downloaded newer image for returntocorp/semgrep:latest
docker.io/returntocorp/semgrep:latest


In [48]:
!docker run -v ./:/opt/ml/code/ returntocorp/semgrep which semgrep

/usr/local/bin/semgrep


In [143]:
!docker run -v ./:/opt/ml/code/ returntocorp/semgrep semgrep scan --registry-caching -q --json --disable-version-check --config=auto --output=/opt/ml/code/scan.json /opt/ml/code/script.py

Error: fatal error
Exiting with error status 2: osemgrep scan --registry-caching -q --json --disable-version-check --config=auto --output=/opt/ml/code/scan.json /opt/ml/code/script.py


In [160]:
import json

with open("scan.json", "r") as json_file:
    scan_results = json.load(json_file)

scan_results.keys()

dict_keys(['errors', 'paths', 'results', 'skipped_rules', 'version'])

In [169]:
print(scan_results["results"][0]["extra"]["metadata"]["cwe"])
print(scan_results["results"][0]["extra"]["metadata"]["impact"])
print(scan_results["results"][0]["extra"]["metadata"])

["CWE-78: Improper Neutralization of Special Elements used in an OS Command ('OS Command Injection')"]
HIGH
{'category': 'security', 'confidence': 'MEDIUM', 'cwe': ["CWE-78: Improper Neutralization of Special Elements used in an OS Command ('OS Command Injection')"], 'cwe2021-top25': True, 'cwe2022-top25': True, 'impact': 'HIGH', 'license': 'Commons Clause License Condition v1.0[LGPL-2.1-only]', 'likelihood': 'MEDIUM', 'owasp': ['A01:2017 - Injection', 'A03:2021 - Injection'], 'references': ['https://owasp.org/www-community/attacks/Command_Injection'], 'semgrep.dev': {'rule': {'origin': 'community', 'rule_id': 'KxUbp2', 'url': 'https://semgrep.dev/playground/r/rxTyL53/python.django.security.injection.command.command-injection-os-system.command-injection-os-system', 'version_id': 'rxTyL53'}}, 'shortlink': 'https://sg.run/Gen2', 'source': 'https://semgrep.dev/r/python.django.security.injection.command.command-injection-os-system.command-injection-os-system', 'subcategory': ['vuln'], 'tec

In [101]:
output = {}
for result in scan_results["results"]:
    line_tuple = f'{result["start"]["line"]}:{result["end"]["line"]}'
    if line_tuple not in output:
        output[line_tuple] = ""
    output[line_tuple] += f'{result["extra"]["message"]}\n\n'
results = []
for out in output:
    results.append({"Start": int(out.split(":")[0]), "End": int(out.split(":")[1]), "Recommendations": output[out]})
    
print(results)

[{'Start': 10, 'End': 13, 'Recommendations': "Request data detected in os.system. This could be vulnerable to a command injection and should be avoided. If this must be done, use the 'subprocess' module instead and pass the arguments as a list. See https://owasp.org/www-community/attacks/Command_Injection for more information.\n\n"}, {'Start': 10, 'End': 18, 'Recommendations': "User-controlled data from a request is passed to 'execute()'. This could lead to a SQL injection and therefore protected information could be leaked. Instead, use django's QuerySets, which are built with query parameterization and therefore not vulnerable to sql injection. For example, you could use `Entry.objects.filter(date=2006)`.\n\n"}, {'Start': 13, 'End': 13, 'Recommendations': "User data detected in os.system. This could be vulnerable to a command injection and should be avoided. If this must be done, use the 'subprocess' module instead and pass the arguments as a list.\n\nFound user-controlled data used 

In [13]:
from sagemaker.tensorflow import TensorFlowModel

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [14]:
import sagemaker

In [15]:
sagemaker.__version__

'2.192.0'